In [2]:
import pandas as pd
import numpy as np
import utils
import swifter
import xgboost
PATH = './data'

In [ ]:
%time train, test = utils.load_data_csv("./data", utils.SIMPLE_FEATURE_COLUMNS + utils.FOI_COLUMNS)

In [3]:
test = pd.read_csv('test_private_v3_track_1.csv.gz')

### Find closest hits

In [ ]:
closest_hits_features_train = train.swifter.apply(
    utils.find_closest_hit_per_station, result_type="expand", axis=1)

In [5]:
closest_hits_features_test = test.swifter.apply(utils.find_closest_hit_per_station, result_type="expand", axis=1)

Pandas Apply: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1452188/1452188 [13:12<00:00, 1833.29it/s]


In [ ]:
train_closest = pd.concat(
    [train.loc[:, utils.SIMPLE_FEATURE_COLUMNS],
     closest_hits_features], axis=1)

In [10]:
test_closest = pd.concat(
    [test.loc[:, utils.SIMPLE_FEATURE_COLUMNS],
     closest_hits_features_test], axis=1)

### Find D^2

In [12]:
# train_closest_isMuon_D = train_closest_isMuon.copy()
test_closest_isMuon_D = test_closest.copy()

In [19]:
# train_closest_isMuon_D = pd.concat(
#     train_closest_isMuon_D,
#      train['FOI_hits_S'], axis=1)
test_closest_isMuon_D = pd.concat([test_closest_isMuon_D, test['FOI_hits_S']], axis = 1)

In [33]:
renamer = dict(zip([i for i in range(24)], [str(i) for i in range(24)]))

In [39]:
test_closest_isMuon_D.rename(columns = renamer, inplace = True)

In [41]:
# train_closest_isMuon_D['D^2'] = utils.calc_D(train_closest_isMuon_D)
test_closest_isMuon_D['D^2'] = utils.calc_D(test_closest_isMuon_D)

### Train model + get predictions

In [ ]:
model = xgboost.XGBClassifier(n_jobs=-1)

In [ ]:
model.fit(train_closest_isMuon_D.values,
              train_closest_isMuon_D.label.values,
              sample_weight=train_closest_isMuon_D.weight.values)

In [ ]:
predictions = model.predict_proba(test_closest_isMuon_D.values)[:, 1]

In [ ]:
pd.DataFrame(data={"prediction": predictions}, index=test_closest_isMuon_D.index).to_csv(
    "sample_submission.csv", index_label=utils.ID_COLUMN)

### Weighted averaging

In [2]:
import pandas as pd
train = pd.read_csv('train_concat_foiS_D2.csv').drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'FOI_hits_S'])

In [4]:
import xgboost

In [ ]:
model_xgb = xgboost.XGBClassifier(n_jobs=-1)
%time model.fit(train.drop(columns = ['label', 'weight']).values,\
              train.label.values, sample_weight=train.weight.values)

In [8]:
test = pd.read_csv('test_concat_foiS_D2.csv')

In [10]:
test.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'FOI_hits_S'], inplace = True)

In [44]:
pred_xgb = model_xgb.predict_proba(test_closest_isMuon_D.drop(columns = 'FOI_hits_S').values)[:, 1]


In [45]:
predictions = pred_xgb

In [46]:
import utils
pd.DataFrame(data={"prediction": predictions}, index=test.index).to_csv(
    "ensemble/final_preds.csv", index_label=utils.ID_COLUMN)